In [1]:
import os
os.chdir(r'C:\Users\半兵卫\Documents\006 项目\项目1 货运放空赔付模型（自动化建模）')

import pandas as pd
import numpy as np
from Base.IV import Iv
from Base.PSI import Psi
from sklearn.utils import shuffle
from Base.ExcelReader import excelReader, getModelVars
from Base.XgBoost import xgBoost
from Base.BivarPlot import bivarPlot
from Base.GetDataSets import getDataSets
from Base.ProcessMissExtreme import processMissExtreme
from Base.PredictScore import predictScore
from Base.CharWoe import charWoe
from IPython.core.interactiveshell import InteractiveShell
from sklearn.model_selection import train_test_split
InteractiveShell.ast_node_interactivity = 'all'

In [2]:
def treatments(datasets):
    for name, pdf in datasets.items():
        # 变量精细化处理
        if "bd_plate_number_fis_record" in pdf.columns:
            pdf[pdf["bd_plate_number_fis_record"] != -100, "bd_plate_number_fis_record"] = \
                [n[0] for n in pdf["bd_plate_number_fis_record"] if n != -100]
            datasets[name] = pdf
    return datasets

#只取出现频率较高的，做截断，其余特征赋值为“其他”
def special(df):
    print("----- bg_cargo_type_name_cargo -----")
    datalis = ['普货', '重货', '设备', '水果', '蔬菜', '配件', '树苗', '泡货', '家具', '西瓜', '建材', '钢材', '石材',
               '化肥', '饮料', '地砖', '木材', '食品', '蜜蜂', '-100']
    lis = list(df["bg_cargo_type_name_cargo"].unique())
    for val in lis:
        if val not in datalis:
            df.loc[df["bg_cargo_type_name_cargo"] == val, "bg_cargo_type_name_cargo"] = u"其他"

    print("----- bg_vehicle_length_name -----")
    df["bg_vehicle_length_name"] = df["bg_vehicle_length_name"].str.strip(u"米")
    df.loc[np.logical_or(df["bg_vehicle_length_name"] == u"不限", df["bg_vehicle_length_name"] == u"不限车长"),
           "bg_vehicle_length_name"] = '-100'
    df["bg_vehicle_length_name"] = pd.to_numeric(df["bg_vehicle_length_name"])

    print("----- sj_plate_number -----")
    df["sj_plate_number"] = df["sj_plate_number"].str[0]

    print("----- bg_vehicle_length_name -----")
    df["sj_vehicle_length"] = pd.to_numeric(df["sj_vehicle_length"].str.strip(u"米"))

    print("----- sj_whitebar_limit -----")
    df["sj_whitebar_limit"] = pd.to_numeric(df["sj_whitebar_limit"].str.strip(u"元"))
    return df



uid, weight, dep = "bg_order_no_vouch", "weight", "bg_result_compensate"

# 数据集处理

In [3]:
df6 = pd.read_csv(r'data\6m.csv',header=0)
df7 = pd.read_csv(r"data\7m.csv", header=0)
df8 = pd.read_csv(r"data\8m.csv", header=0)
df = pd.concat([df6,df7,df8])
df["weight"] = 4
df.loc[df[dep] == 1, weight] = 1
df = df.sample(50000)
df.index = range(df.shape[0])

offf = pd.read_csv(r"data\9m.csv", header=0)
offf["weight"] = 4
offf.loc[offf[dep] == 1, weight] = 1
offf = offf.sample(10000)
offf.index = range(offf.shape[0])

#切分训练集和测试集
devf = df[df.index % 10 < 6]
valf = df[df.index % 10 >= 6]
devf.to_csv(r"data\dev_raw_data.csv", index=None)
valf.to_csv(r"data\val_raw_data.csv", index=None)
offf.to_csv(r"data\off_raw_data.csv", index=None)
devf.shape
valf.shape
offf.shape

D:\python3\anconda\lib\site-packages\IPython\core\interactiveshell.py:3063: DtypeWarning: Columns (7,9) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
D:\python3\anconda\lib\site-packages\IPython\core\interactiveshell.py:3063: DtypeWarning: Columns (7,9,46,96,98) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


(30000, 201)

(20000, 201)

(10000, 201)

# 特殊字段处理

In [7]:
getModelVars(path=r"data\dev_raw_data.csv",     #数值型变量取出来，导入到model_trace中
             dep=dep, 
             weight=weight,
             outfile=r'report\model_trace.xlsx')

devf = pd.read_csv(r"data\dev_raw_data.csv", header=0)
dev = special(devf)
dev.to_csv(r"data\dev_raw_data_special.csv", index=None)
del devf
del dev
valf = pd.read_csv(r"data\val_raw_data.csv", header=0)
val = special(valf)
val.to_csv(r"data\val_raw_data_special.csv", index=None)
del valf
del val
offf = pd.read_csv(r"data\off_raw_data.csv", header=0)
off = special(offf)
off.to_csv(r"data\off_raw_data_special.csv", index=None)
del offf
del off
#在model_trace文件中，
#删除"bg_"开头的特征；
#"hz_user_id"和"hz_domain_id"。

----- bg_cargo_type_name_cargo -----
----- bg_vehicle_length_name -----
----- sj_plate_number -----
----- bg_vehicle_length_name -----
----- sj_whitebar_limit -----
----- bg_cargo_type_name_cargo -----
----- bg_vehicle_length_name -----
----- sj_plate_number -----
----- bg_vehicle_length_name -----
----- sj_whitebar_limit -----
----- bg_cargo_type_name_cargo -----
----- bg_vehicle_length_name -----
----- sj_plate_number -----
----- bg_vehicle_length_name -----
----- sj_whitebar_limit -----


In [6]:
# 字符串woe处理
var_names = excelReader(file=r"report\model_trace.xlsx",   #读取model_trace中的字段名
                        sheet="vars", 
                        var_names=[]).getcols()  
print("var_names: %s" % len(var_names))
vars = [
     "hz_cargo_type_name",
     "hz_city_name",
     "hz_departure_name",
     "hz_destination_name",
     "hz_vehicle_type_name",
     "sj_30day_auth",
     "sj_30day_register",
     "sj_7day_auth",
     "sj_7day_register",
     "sj_card_name",
     "sj_city_name",
     "sj_etc_open",
     "sj_vehicle_type",
     "sj_vip_type",
     "sj_p_cargo_truck_type",
     "sj_plate_city_name",
     "sj_plate_number",
     "sj_plate_province_name"
]

datasets = getDataSets(devp=r'data\dev_raw_data_special.csv',
                        valp=r'data\val_raw_data_special.csv',
                        offp=r'data\off_raw_data_special.csv',
                        usecols=var_names+[uid, weight, dep]+vars  #导入var_names,vars,uid,dep,weight
                      ).loading()  
datasets = treatments(datasets)

print("loading dataset is ok...")
datasets = charWoe(datasets=datasets, 
                   keep_vars=[uid, weight, dep], 
                   weight=weight, 
                   dep=dep, 
                   vars=vars).char_woe()

datasets.get("dev", "").to_csv(r"data\dev_raw_data_special_woe.csv", index=None)
datasets.get("val", "").to_csv(r"data\val_raw_data_special_woe.csv", index=None)
datasets.get("off", "").to_csv(r"data\off_raw_data_special_woe.csv", index=None)

FileNotFoundError: [Errno 2] No such file or directory: 'report\\model_trace.xlsx'

# 缺失值、极值处理

In [10]:
# step4: 数据缺失值、极值处理&数据集转换(不再作此处理)
print("var_names: %s" % len(var_names))
#对数值型变量做woe处理
processMissExtreme(devp=r"data\dev_raw_data_special_woe.csv",
                   valp=r"data\val_raw_data_special_woe.csv",
                   offp=r"data\off_raw_data_special_woe.csv",
                   var_names=var_names).dealing()

var_names: 142
0 sj_day_auth -100.0 969.0
1 sj_day_register -100.0 1431.0
2 sj_age -100.0 54.0
3 sj_load -100.0 50.0
4 sj_cnt_ht_empty -100.0 1.0
5 sj_amt_ht_empty -100.0 5.0
6 sj_cnt_compensate -100.0 0.0
7 sj_cnt_compensate_suceess -100.0 0.0
8 sj_cnt_vouch_driver -100.0 2.0
9 sj_p_cargo_line_length -100.0 2342.01
10 sj_p_cargo_truck_length -100.0 17.0
11 sj_p_cargo_truck_load -100.0 100.0
12 sj_day_fis_vouch_rate -100.0 1.0
13 sj_30_days_cnt_ht_empty -100.0 16.0
14 sj_30_days_amt_ht_empty -100.0 80.0
15 sj_30_days_cnt_compensate -100.0 2.0
16 sj_30_days_cnt_compensate_suceess -100.0 1.0
17 sj_30_days_cnt_compensate_rate -100.0 0.5
18 sj_30_days_cnt_compensate_suceess_rate -100.0 0.3333
19 sj_60_days_cnt_ht_empty -100.0 28.0
20 sj_60_days_amt_ht_empty -100.0 140.0
21 sj_60_days_cnt_compensate -100.0 2.0
22 sj_60_days_cnt_compensate_suceess -100.0 2.0
23 sj_60_days_cnt_compensate_rate -100.0 0.5
24 sj_60_days_cnt_compensate_suceess_rate -100.0 0.3333
25 sj_90_days_cnt_ht_empty -100.0 

In [ ]:
# 训练模型（使用特征重要性迭代删除变量）

#取列名，放入model_trace.xlsx中
#先将以前的mdel_trace删除,再做getmodelvars
getModelVars(path=r"data\off_raw_data_special_woe_trt.csv", dep=dep, weight=weight,
             outfile=r'report\model_trace.xlsx')
var_names = excelReader(file=r"report\model_trace.xlsx", sheet="vars", var_names=[]).getcols()
print("var_names: %s" % len(var_names))

#删除bg开头的特征
var_names = excelReader(file=r"report\model_trace.xlsx", sheet="vars", var_names=[]).getcols()
print("var_names: %s" % len(var_names))

datasets = getDataSets(devp=r'data\dev_raw_data_special_woe_trt.csv',
                       valp=r'data\val_raw_data_special_woe_trt.csv',
                       offp=r'data\off_raw_data_special_woe_trt.csv',
                       usecols=var_names + [uid, weight, dep]).loading()

# xgboost 算法训练
#生成model.pkl
#生成Xgboost_train_tmp
#生成ks报告
params = {
    "max_depth": 2,
    "learning_rate": 0.09,
    "n_estimators": 120,
    "min_child_weight": 50,
    "subsample": 1,
    "scale_pos_weight": 1,
    "reg_lambda": 30
}
xgBoost(datasets=datasets,
        uid=uid, 
        dep=dep, 
        weight=weight, 
        var_names=var_names, 
        params=params,
        max_del_var_nums=20).training(min_score=0.02, 
                             modelfile=r"report\model_report.pmml", 
                             output_scores=[])

# 计算IV、psi和bivar

In [15]:
#step6: 计算变量IV
var_names = excelReader(file=r"report\model_trace.xlsx", sheet="vars", var_names=[]).getcols()
print("var_names: %s" % len(var_names))
Iv(datapath=r'data\dev_raw_data_special_woe_trt.csv', 
   dep=dep, 
   var_names=var_names, 
   outfile=r'report\IV.xlsx').sloveIv()

var_names: 22
0 hz_60_days_cnt_compensate_suceess 7
1 hz_90_days_cnt_compensate_suceess_rate 8
2 hz_avg_cargo_call_times_last30d_rate 8
3 sj_60_days_cnt_compensate_rate 8
4 sj_90_days_cnt_compensate_rate 8
5 hz_30_days_cnt_compensate_suceess 5
6 sj_60_days_cnt_compensate_suceess_rate 6
7 hz_60_days_cnt_compensate_rate 8
8 sj_90_days_cnt_compensate 6
9 hz_90_days_cnt_compensate_suceess 7
10 sj_90_days_cnt_compensate_suceess_rate 8
11 sj_plate_number_woe 8
12 si_fis_vouch_90day_rate 8
13 hz_90_days_cnt_compensate_rate 8
14 hz_30_days_cnt_compensate_rate 8
15 hz_60_days_cnt_compensate_suceess_rate 8
16 hz_30_days_cnt_cargo_fis 8
17 hz_30_days_cnt_cargo_compensate 7
18 sj_vehicle_type_woe 5
19 hz_30_days_cnt_vouch_consignor 8
20 sj_card_name_woe 5
21 sj_plate_province_name_woe 7


In [16]:
var_names = excelReader(file=r"report\model_trace.xlsx", sheet="vars", var_names=[]).getcols()
print("var_names: %s" % len(var_names))
Psi(datasets=datasets, dep=dep, var_names=var_names, ivfile=r'report\IV.xlsx', outfile=r"report\PSI.xlsx").slove()

var_names: 22
0 hz_60_days_cnt_compensate_suceess
1 hz_90_days_cnt_compensate_suceess_rate
2 hz_avg_cargo_call_times_last30d_rate
3 sj_60_days_cnt_compensate_rate
4 sj_90_days_cnt_compensate_rate
5 hz_30_days_cnt_compensate_suceess
6 sj_60_days_cnt_compensate_suceess_rate
7 hz_60_days_cnt_compensate_rate
8 sj_90_days_cnt_compensate
9 hz_90_days_cnt_compensate_suceess
10 sj_90_days_cnt_compensate_suceess_rate
11 sj_plate_number_woe
12 si_fis_vouch_90day_rate
13 hz_90_days_cnt_compensate_rate
14 hz_30_days_cnt_compensate_rate
15 hz_60_days_cnt_compensate_suceess_rate
16 hz_30_days_cnt_cargo_fis
17 hz_30_days_cnt_cargo_compensate
18 sj_vehicle_type_woe
19 hz_30_days_cnt_vouch_consignor
20 sj_card_name_woe
21 sj_plate_province_name_woe


In [17]:
var_names = excelReader(file=r"report\model_trace.xlsx", sheet="vars", var_names=[]).getcols()
print("var_names: %s" % len(var_names))
bivarPlot(datasets=datasets, dep=dep, var_names=var_names, ivfile=r'report\IV.xlsx', outfile=r"report\Bivar.xlsx").plot()

var_names: 21
0 sj_vehicle_type_woe
1 sj_60_days_cnt_compensate_suceess_rate
2 sj_card_name_woe
3 sj_90_days_cnt_compensate_suceess_rate
4 hz_30_days_cnt_cargo_fis
5 sj_plate_province_name_woe
6 hz_30_days_cnt_vouch_consignor
7 sj_plate_number_woe
8 sj_60_days_cnt_compensate_rate
9 si_fis_vouch_90day_rate
10 sj_90_days_cnt_compensate_rate
11 hz_30_days_cnt_compensate_rate
12 hz_60_days_cnt_compensate_rate
13 hz_60_days_cnt_compensate_suceess_rate
14 hz_60_days_cnt_compensate_suceess
15 hz_30_days_cnt_cargo_compensate
16 hz_90_days_cnt_compensate_rate
17 hz_30_days_cnt_compensate_suceess
18 hz_90_days_cnt_compensate_suceess_rate
19 sj_90_days_cnt_compensate
20 hz_90_days_cnt_compensate_suceess


In [18]:
var_names = excelReader(file=r"report\model_trace.xlsx", sheet="vars", var_names=[]).getcols()
print("var_names: %s" % len(var_names))

var_names: 21


# 自动化调参

In [19]:
# step7: 参数自动化调整
var_names = excelReader(file=r"report\model_trace.xlsx", sheet="vars", var_names=[]).getcols()
print("var_names: %s" % len(var_names))

xgBoost(datasets=datasets, uid=uid, dep=dep, weight=weight, var_names=var_names,
         params=dict(), max_del_var_nums=0).auto_choose_params(target="weight")

var_names: 21
train_number: 0, devks: 0.38860815761457407, offks: 0.3744819536991994, params: {'max_depth': 5, 'learning_rate': 0.09, 'n_estimators': 120, 'min_child_weight': 50, 'subsample': 1, 'scale_pos_weight': 1, 'reg_lambda': 21}
(Bad) train_number: 1, devks: 0.4074215322513981, offks: 0.3708627002892385, params: {'max_depth': 5, 'learning_rate': 0.14, 'n_estimators': 120, 'min_child_weight': 50, 'subsample': 1, 'scale_pos_weight': 1, 'reg_lambda': 21}
(Good) train_number: 2, devks: 0.37315671828602026, offks: 0.3710267404281636, params: {'max_depth': 5, 'learning_rate': 0.04000000000000001, 'n_estimators': 120, 'min_child_weight': 50, 'subsample': 1, 'scale_pos_weight': 1, 'reg_lambda': 21}
(Bad) train_number: 3, devks: 0.4052449160362829, offks: 0.3710814204744719, params: {'max_depth': 6, 'learning_rate': 0.09000000000000001, 'n_estimators': 120, 'min_child_weight': 50, 'subsample': 1, 'scale_pos_weight': 1, 'reg_lambda': 21}
(Bad) train_number: 4, devks: 0.3867557733005915, o

In [20]:
params = {'max_depth': 5, 'learning_rate': 0.09000000000000001, 
          'n_estimators': 120, 'min_child_weight': 100, 
          'subsample': 1.0, 'scale_pos_weight': 1, 'reg_lambda': 21}

# 递归特征删除（使用ks）

In [21]:
# step8: 变量自动化调整(是否可以删除一些无用的变量)
xgBoost(datasets=datasets, uid=uid, dep=dep, weight=weight,
        var_names=var_names, params=params, max_del_var_nums=1).auto_delete_vars()

train_number: 0, offks: 0.3768511081194213
(Bad) train_n: 1, offks: 0.37602336534944314 by vars: 21
(Good) train_n: 2, offks: 0.3815196527628508 by vars: sj_60_days_cnt_compensate_suceess_rate
(Good) train_n: 3, offks: 0.3828706270104345 by vars: sj_card_name_woe
(Bad) train_n: 4, offks: 0.3807324486478944 by vars: 19
(Bad) train_n: 5, offks: 0.3734336052251498 by vars: 19
(Bad) train_n: 6, offks: 0.379426726852428 by vars: 19
(Bad) train_n: 7, offks: 0.3785659875027812 by vars: 19
(Bad) train_n: 8, offks: 0.37923157565267235 by vars: 19
(Bad) train_n: 9, offks: 0.3761826916912727 by vars: 19
(Bad) train_n: 10, offks: 0.3692977196535171 by vars: 19
(Bad) train_n: 11, offks: 0.3761402675174128 by vars: 19
(Bad) train_n: 12, offks: 0.37541434276469854 by vars: 19
(Bad) train_n: 13, offks: 0.381315073968904 by vars: 19
(Bad) train_n: 14, offks: 0.38216167192726425 by vars: 19
(Bad) train_n: 15, offks: 0.3816554101192025 by vars: 19
(Good) train_n: 16, offks: 0.3873393066570128 by vars: hz

In [22]:
del_list = ['sj_60_days_cnt_compensate_suceess_rate',
            'sj_card_name_woe',
            'hz_30_days_cnt_cargo_compensate']

for i in del_list:
    var_names.remove(i)
print("var_names: %s" % len(var_names))

var_names_end = {'变量': var_names}
pd.DataFrame(var_names_end).to_csv(r"report\var_names_end.csv", index=None)

var_names: 18


# 重新拟合模型并且生成报告

In [25]:
#生成报告
var_names = excelReader(file=r"report\model_trace.xlsx", sheet="vars", var_names=[]).getcols()
print("var_names: %s" % len(var_names))

params = {'max_depth': 5, 'learning_rate': 0.09000000000000001, 
          'n_estimators': 120, 'min_child_weight': 50, 
          'subsample': 1.0, 'scale_pos_weight': 1, 'reg_lambda': 21}

datasets = getDataSets(devp=r'data\dev_raw_data_special_woe_trt.csv',
                       valp=r'data\val_raw_data_special_woe_trt.csv',
                       offp=r'data\off_raw_data_special_woe_trt.csv',
                       usecols=var_names + [uid, weight, dep]).loading()

var_names: 18


In [26]:
xgBoost(datasets=datasets,
        uid=uid, 
        dep=dep, 
        weight=weight, 
        var_names=var_names, 
        params=params,
        max_del_var_nums=0).training(min_score=0.02, 
                             modelfile=r"report\model_report.pmml", 
                             output_scores=[])

del var:  18 --> 18 ks:  {'devks': 0.39730211031008233, 'valks': 0.3340299496763066, 'offks': 0.37401340226790203, 'valpsi': 0.0009791947578007745, 'offpsi': 0.0032080513949902622} 
[331.0, 182.0, 117.0, 94.0, 71.0, 58.0, 44.0, 39.0, 52.0, 43.0, 36.0, 39.0, 26.0, 30.0, 28.0, 13.0, 18.0, 17.0, 12.0, 1.0]
dev ks: 0.3918657414671415
[169.0, 99.0, 74.0, 57.0, 35.0, 38.0, 46.0, 36.0, 24.0, 21.0, 22.0, 21.0, 19.0, 15.0, 22.0, 27.0, 20.0, 9.0, 14.0, 8.0]
val ks: 0.32202015521834815
[95.0, 53.0, 45.0, 37.0, 38.0, 21.0, 26.0, 14.0, 9.0, 16.0, 10.0, 10.0, 9.0, 16.0, 10.0, 8.0, 7.0, 8.0, 10.0, 2.0]
off ks: 0.36506850089939247


In [33]:
path = r'C:\Users\半兵卫\Documents\006 项目\项目1 货运放空赔付模型（自动化建模）'
import joblib
model = joblib.load(path + './model.pkl') 
model

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
              importance_type='gain', interaction_constraints='',
              learning_rate=0.09000000000000001, max_delta_step=0, max_depth=5,
              min_child_weight=50, missing=None, monotone_constraints='()',
              n_estimators=120, n_jobs=10, nthread=10, num_parallel_tree=1,
              random_state=7, reg_alpha=0, reg_lambda=21, scale_pos_weight=1,
              subsample=1.0, tree_method='exact', validate_parameters=1,
              verbosity=None)

In [34]:
#打分code
model.predict_proba(datasets['off'][var_names])[:,1]

array([0.03121824, 0.02948882, 0.02666607, ..., 0.06446788, 0.2195385 ,
       0.01402299], dtype=float32)

In [24]:
#生成报告
xgBoost(datasets=datasets, uid=uid, dep=dep, weight=weight,
        var_names=var_names, params=params, max_del_var_nums=0).plotKS(model, bins=15)

[1108.0, 22.0, 16.0, 23.0, 4.0, 3.0, 2.0, 1.0, 2.0, 0, 0, 0, 0, 0, 0]
dev ks: 0.8857572882035587
[191.0, 87.0, 85.0, 61.0, 54.0, 45.0, 44.0, 50.0, 34.0, 36.0, 31.0, 25.0, 23.0, 20.0, 20.0]
val ks: 0.2635741473180593
[87.0, 47.0, 41.0, 33.0, 18.0, 26.0, 25.0, 23.0, 22.0, 19.0, 13.0, 19.0, 12.0, 10.0, 5.0]
off ks: 0.2571875
